In [61]:
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

%matplotlib  inline               
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import folium
import folium.plugins 

#pd.set_option('html', False)  
# unknown error, skip this step
pd.set_option('max_columns', 30)
pd.set_option('max_rows', 20)

## 1. Based on organzied csv files, get a table about all vehicle collisions from 2014 to 2017

### 1.1 combine vehicle collisions csv files from 2014 to 2017 (in the order of time)

In [2]:
df_csv1417 = pd.DataFrame()

for y in range(2014, 2018):
    fname = "collision%d.csv" % (y)
    tmp_df = pd.read_csv(fname)
    tmp_df["Year"] = y
    df_csv1417 = df_csv1417.append(tmp_df, sort = False)

df_csv1417.index = range(df_csv1417.shape[0])
df_csv1417["Date"] = pd.to_datetime(df_csv1417["Date"])

df_csv1417["Month"] = df_csv1417["Date"].apply(lambda x: x.month)

def DayOfWeek(date):
    dw = date.weekday()
    rst = {
        0: 'T1_Mon',
        1: 'T2_Tue',
        2: 'T3_Wed',
        3: 'T4_Thu',
        4: 'T5_Fri',
        5: 'T6_Sat',
        6: 'T7_Sun'
    }[dw]
    return rst

df_csv1417["Day_Of_Week"] = df_csv1417["Date"].apply(lambda x: DayOfWeek(x))

df_csv1417["Environment"] = df_csv1417["Environment"].astype("category")
df_csv1417["Road_Surface"] = df_csv1417["Road_Surface"].astype("category")
df_csv1417["Traffic_Control"] = df_csv1417["Traffic_Control"].astype("category")
df_csv1417["Collision_Location"] = df_csv1417["Collision_Location"].astype("category")
df_csv1417["Light"] = df_csv1417["Light"].astype("category")
df_csv1417["Collision_Classification"] = df_csv1417["Collision_Classification"].astype("category")
df_csv1417["Impact_type"] = df_csv1417["Impact_type"].astype("category")

df_csv1417

,Record,Location,Date,Time,Environment,Road_Surface,Traffic_Control,Collision_Location,Light,Collision_Classification,Impact_type,longitude,latitude,Year,Month,Day_Of_Week
0,2014000001,RIDEAU ST @ WALLER ST,2014-02-21,06:07:00,02 - Rain,02 - Wet,01 - Traffic signal,03 - At intersection,07 - Dark,01 - Fatal injury,07 - SMV other,-75.688726,45.427533,2014,2,T5_Fri
1,2014000002,HINES RD btwn INNOVATION DR & SOLANDT RD,2014-08-02,13:55:00,01 - Clear,01 - Dry,10 - No control,04 - At/near private drive,01 - Daylight,01 - Fatal injury,02 - Angle,-75.921033,45.343152,2014,8,T6_Sat
2,2014000003,LOGGERS WAY btwn KINGDON MINE RD & GALETTA SID...,2014-06-20,23:15:00,01 - Clear,01 - Dry,10 - No control,01 - Non intersection,07 - Dark,01 - Fatal injury,07 - SMV other,-76.247045,45.438627,2014,6,T5_Fri
3,2014000004,MONTREAL RD @ BATHGATE DR/BURMA RD,2014-07-29,16:31:00,01 - Clear,01 - Dry,01 - Traffic signal,03 - At intersection,01 - Daylight,01 - Fatal injury,05 - Turning movement,-75.625271,45.447248,2014,7,T2_Tue
4,2014000005,RICHMOND RD btwn FALLOWFIELD RD & RUSHMORE RD,2014-08-05,20:05:00,01 - Clear,01 - Dry,10 - No control,01 - Non intersection,05 - Dusk,01 - Fatal injury,07 - SMV other,-75.820252,45.251073,2014,8,T2_Tue
5,2014000006,PARKDALE AVE btwn RUSKIN ST & INGLEWOOD PL,2014-04-19,10:22:00,01 - Clear,01 - Dry,10 - No control,01 - Non intersection,01 - Daylight,01 - Fatal injury,06 - SMV unattended vehicle,-75.724021,45.392357,2014,4,T6_Sat
6,2014000007,CARLING AVE/STATION RD @ MARCH RD,2014-07-27,13:10:00,01 - Clear,01 - Dry,01 - Traffic signal,02 - Intersection related,01 - Daylight,01 - Fatal injury,99 - Other,-75.911687,45.338652,2014,7,T7_Sun
7,2014000008,CATHERINE ST @ KENT ST,2014-08-12,22:18:00,02 - Rain,02 - Wet,01 - Traffic signal,03 - At intersection,07 - Dark,01 - Fatal injury,07 - SMV other,-75.693767,45.408797,2014,8,T2_Tue
8,2014000009,DOYLE RD @ RIVER RD,2014-09-06,10:58:00,02 - Rain,02 - Wet,02 - Stop sign,02 - Intersection related,01 - Daylight,01 - Fatal injury,04 - Sideswipe,-75.622802,45.171309,2014,9,T6_Sat
9,2014000010,KATIMAVIK RD btwn PEARY WAY & CHIMO DR,2014-04-22,16:17:00,01 - Clear,01 - Dry,10 - No control,01 - Non intersection,01 - Daylight,01 - Fatal injury,07 - SMV other,-75.886024,45.314923,2014,4,T2_Tue


### 1.2 append collision table with daily weather

In [3]:
wt1417 = pd.read_csv("weather1417.csv")

casesByDay = df_csv1417.groupby(['Date']).size().tolist()
wt1417["Number of Collisions"] = casesByDay

wt1417["Date"] = pd.to_datetime(wt1417["Date"])

wt1417["Year"] = wt1417["Date"].apply(lambda x: x.year)
wt1417["Month"] = wt1417["Date"].apply(lambda x: x.month)
wt1417["Day in month"] = wt1417["Date"].apply(lambda x: x.day)

wt1417

,Date,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Number of Collisions,Year,Month,Day in month
0,2014-01-01,-19.2,-23.0,-21.1,0.0,0.0,0.0,30,31,2014,1,1
1,2014-01-02,-20.8,-27.1,-24.0,0.0,0.0,0.0,29,74,2014,1,2
2,2014-01-03,-21.0,-28.1,-24.6,0.0,0.3,0.0,29,151,2014,1,3
3,2014-01-04,-8.5,-27.5,-18.0,0.0,7.5,2.6,29,51,2014,1,4
4,2014-01-05,5.8,-12.0,-3.1,13.2,5.5,18.6,36,52,2014,1,5
5,2014-01-06,5.8,-8.2,-1.2,0.0,0.0,0.0,31,36,2014,1,6
6,2014-01-07,-14.0,-17.9,-16.0,0.0,0.0,0.0,29,69,2014,1,7
7,2014-01-08,-9.5,-16.4,-13.0,0.0,0.0,0.0,29,71,2014,1,8
8,2014-01-09,-7.6,-14.7,-11.2,0.0,0.5,0.2,29,74,2014,1,9
9,2014-01-10,-1.5,-15.1,-8.3,0.0,0.0,0.0,29,36,2014,1,10


Date
2014-01-01     31
2014-01-02     74
2014-01-03    151
2014-01-04     51
2014-01-05     52
2014-01-06     36
2014-01-07     69
2014-01-08     71
2014-01-09     74
2014-01-10     36
             ... 
2017-12-22     57
2017-12-23     75
2017-12-24     28
2017-12-25     31
2017-12-26     44
2017-12-27     62
2017-12-28     73
2017-12-29     63
2017-12-30     47
2017-12-31     43
Length: 1461, dtype: int64

### 1.3 mark holdays

## 2. Based on the table we get, do 

### 2.1 Overview

In [4]:
countOfCollision = df_csv1417.groupby(['Year']).size()
growthRate = (countOfCollision.pct_change()*100).apply(lambda x: '' if pd.isna(x) else format(x,'.2f') + '%')
barOfYear = go.Bar(x = range(2014,2018), 
                   y = countOfCollision,
                   text = countOfCollision,
                   textposition = 'auto',
                   opacity=0.6, 
                   name = 'Number of vehicle collisions in year')
scatterOfYear = go.Scatter(x = range(2014,2018), 
                           y = countOfCollision,
                           text = growthRate,
                           mode = 'lines+text+markers',
                           textposition = 'top center',
                           name = 'Growth rate of vehicle collisions (compared with the previous year)')

fig = dict(data = [barOfYear,scatterOfYear], 
           layout = go.Layout(title = 'Number of vehicle collisions in year', xaxis=dict(title='Year', dtick=1)))
iplot(fig)

print("Total number of cases: " + str(countOfCollision.sum()))

Total number of cases: 58338


**Brief Conclusion:**
* From January 1, 2014 to December 31, 2017, there were **58338** vehicle collision cases happened in Ottawa. 
* The differences of vehicle collision cases during 4 years are small. 
* There is a relatively big decrease of the number of cases between 2015 and 2016(-7.0%), while the rest of growth rate between years are increase. 

**Hypothesis:**
* There's no relationship between year and vehicle collisions, or to say driving in a specific year does not lead to a vehicle collision. 

### 2.2 Percentage of single possible causes of accident

In [5]:
def analysisInSinglePercent(Col): 
    count = '# of Cases'
    df_cause = pd.crosstab(index = df_csv1417[Col], columns = count, margins = False)
    df_cause[Col] = df_cause.index
    sumOf = df_cause[count].sum()
    df_cause['Percentage'] = df_cause[count]/sumOf*100
    df_cause = df_cause[[Col, count, 'Percentage']]
    df_cause = df_cause.sort_values([count], ascending=[0])
    
    tab = go.Table(header = dict(values = ['<b>' + a + '</b>' for a in df_cause.columns.tolist()]),
                   domain = dict(x=[0, 0.4], y=[0,1.0]),
                   columnwidth = [110,55,55], 
                   cells = dict(values = [df_cause[b].tolist() for b in df_cause.columns.tolist()],
                                font = dict(color=['rgb(40, 40, 40)'] * 5, size=12),
                                format = [None] + [None] + ['.2f'],
                                suffix = ["", "", "%"], 
                                align = ["left", "right", "right"]),
                  )
    pie = go.Pie(values = df_cause[count], 
                 labels = df_cause.index, 
                 name = df_cause.index.name,
                 domain=dict(x=[0.6, 1.0], y=[0, 1.0]),
                 hoverinfo = "label+percent+name", hole = .4, showlegend = True)
    fig = dict(data = [tab,pie], 
               layout = go.Layout(title = Col))
    iplot(fig)
    
for n in ['Environment', 'Road_Surface', 'Traffic_Control', 'Collision_Location', \
          'Light', 'Collision_Classification', 'Impact_type', 'Month', "Day_Of_Week"]:
    analysisInSinglePercent(n) 

**Brief Conclusion - Most of car collisions happen in:**
* environment: clear (78.73%)
* road surface: dry (65.76%)
* traffic control: no traffic control (47.97%)
* collision location: at intersection or intersection related (21.47%+31.34% = 52.81%)
* light condition: daylight (68.36%)
* collision classification: P.D. only (80.99%)
* impact type: varies
* month: December, January, February (11.14%+10.37%+9.90% = 31.41%)
* day of week: workdays, the trend keeps rising from Monday to Friday

**Hypothesis:**
* Those weather that seems common, such as clear environment and dry road surface might lead drivers to be careless(it might be not a dependable factor: the proportions of those weathers such as clear days and dry road surfaces is much more bigger than the proportions of wet road surface and snowy environment). 
* A place without traffic control might lead drivers to ignore some details that perhaps cause accidents, or some drivers could break the rules. 
* The location where is close to of at intersection requires drivers to control their wheels and gases carefully due to the heavy traffic and roads to different directions. Under this kind of stressful situation, people without enough driving experience are easy to make mistakes. 
* Of course, most of collisions did not cause injuries, and the impact type varies, either. P.D. only collisions are highly related to SMV other and rear end impacts. 
* December, January and February are in winter of Ottawa. The season of winter in Ottawa is cold, so that this season causes many potential factors of collision: snow and ice creates terrible road surface condition that let cars slip, low temperature makes drivers' reaction slower, snow and long night reduces their visability, etc. 
* Objectively, the traffic is heavier on workdays in a week. According to the table, the stress of people on workdays increases step by step, and the proportion of collisions is also divided by days of week step by step. 

### 2.3 Temperature & Weather Related Variables

In [6]:
df_avg = wt1417.groupby(["Year","Month"]).mean()
df_avg["Year"] = [x[0] for x in df_avg.index]
df_avg["Month"] = [x[1] for x in df_avg.index]

df_sum = wt1417.groupby(["Year","Month"]).sum()
df_sum["Year"] = [x[0] for x in df_sum.index]
df_sum["Month"] = [x[1] for x in df_sum.index]

def printTemInYear(n):
    monthCase = go.Bar(x = df_sum[df_sum["Year"]==n]["Month"],
                       y = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       text = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       textposition = 'auto',
                       opacity = 0.6,
                       name = 'Number of Collisions')
    max_temp = go.Scatter(x = df_avg[df_avg["Year"]==n]["Month"],
                         y = df_avg[df_avg["Year"]==n]["Max Temp (°C)"],
                         mode = 'lines',
                         name = 'Monthly Mean Max Temp',
                         xaxis = 'x2', 
                         yaxis = 'y2')
    min_temp = go.Scatter(x = df_avg[df_avg["Year"]==n]["Month"],
                         y = df_avg[df_avg["Year"]==n]["Min Temp (°C)"],
                         mode = 'lines',
                         name = 'Monthly Mean Min Temp',
                         fill = 'tonexty', 
                         xaxis = 'x2', 
                         yaxis = 'y2')
    fig = dict(data = [monthCase, max_temp,min_temp], 
               layout = go.Layout(title = "Temperature Analysis (" + str(n) + ")",
                                  xaxis = dict(title='Month', dtick=1),
                                  yaxis = dict(title='# of Collisions', dtick=500, domain = [0,0.5]),
                                  yaxis2 = dict(title='Temp (°C)', dtick=5, domain = [0.55,1.0],autorange='reversed'),
                                  xaxis2 = dict(dtick=1, anchor = 'y2')
                                 )
              )
    iplot(fig)
    
for m in range(2014,2018):
    printTemInYear(m)
    
print("The correlation between number of collisions and mean temperature: \n" 
      + "all months = %.4f" % df_avg["Number of Collisions"].corr(df_avg["Mean Temp (°C)"]) + "\n" 
      + "In each month: " + "\n")

def corrInMonth(l):
      s = {
        1: 'January',
        2: 'February',
        3: 'March',
        4: 'April',
        5: 'May',
        6: 'June',
        7: 'July',
        8: 'August',
        9: 'September',
       10: 'October',
       11: 'November',
       12: 'December'
      }[l]
      print(s + " = %.4f" % df_avg[df_avg["Month"] == l]["Number of Collisions"].corr(df_avg["Mean Temp (°C)"]))
        
for f in range(1,13):
    corrInMonth(f)

The correlation between number of collisions and mean temperature: 
all months = -0.7044
In each month: 

January = -0.9936
February = -0.9209
March = -0.7807
April = -0.8263
May = 0.9109
June = -0.4081
July = -0.0211
August = -0.5915
September = 0.3481
October = -0.1837
November = -0.7828
December = -0.9908


**Brief Conclusion:**
* The number of collisions in winter is always higher than other months in a year. 
* The correlation between the number of collisions and month becomes extremely low in winter. 

**Hypothesis:**
* Extreme low temperature leads to vehicle collisions, such as vehicle faults and several weathers that are related to low temperature. 

In [7]:
df_avg = wt1417.groupby(["Year","Month"]).mean()
df_avg["Year"] = [x[0] for x in df_avg.index]
df_avg["Month"] = [x[1] for x in df_avg.index]

df_sum = wt1417.groupby(["Year","Month"]).sum()
df_sum["Year"] = [x[0] for x in df_sum.index]
df_sum["Month"] = [x[1] for x in df_sum.index]

def printRainInYear(n):
    monthCase = go.Bar(x = df_sum[df_sum["Year"]==n]["Month"],
                       y = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       text = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       textposition = 'auto',
                       opacity = 0.6,
                       name = 'Number of Collisions')
    total_rain = go.Scatter(x = df_sum[df_sum["Year"]==n]["Month"],
                         y = df_sum[df_sum["Year"]==n]["Total Rain (mm)"],
                         mode = 'lines',
                         name = 'Monthly Total Rain',
                         xaxis = 'x2', 
                         yaxis = 'y2')
    fig = dict(data = [monthCase, total_rain], 
               layout = go.Layout(title = "Rain Analysis (" + str(n) + ")",
                                  xaxis = dict(title='Month', dtick=1),
                                  yaxis = dict(title='# of Collisions', dtick=500, domain = [0,0.5]),
                                  yaxis2 = dict(title='Total Rain (mm)', domain = [0.55,1.0]),
                                  xaxis2 = dict(dtick=1, anchor = 'y2')
                                 )
              )
    iplot(fig)
    
for m in range(2014,2018):
    printRainInYear(m)

**Brief Conclusion:**
* No clear relationship between rain and collisions

In [8]:
df_avg = wt1417.groupby(["Year","Month"]).mean()
df_avg["Year"] = [x[0] for x in df_avg.index]
df_avg["Month"] = [x[1] for x in df_avg.index]

df_sum = wt1417.groupby(["Year","Month"]).sum()
df_sum["Year"] = [x[0] for x in df_sum.index]
df_sum["Month"] = [x[1] for x in df_sum.index]

def printSnowInYear(n):
    monthCase = go.Bar(x = df_sum[df_sum["Year"]==n]["Month"],
                       y = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       text = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       textposition = 'auto',
                       opacity = 0.6,
                       name = 'Number of Collisions')
    total_snow = go.Scatter(x = df_sum[df_sum["Year"]==n]["Month"],
                         y = df_sum[df_sum["Year"]==n]["Total Snow (cm)"],
                         mode = 'lines',
                         name = 'Monthly Total Snow',
                         xaxis = 'x2', 
                         yaxis = 'y2')
    fig = dict(data = [monthCase, total_snow], 
               layout = go.Layout(title = "Snow Analysis (" + str(n) + ")",
                                  xaxis = dict(title='Month', dtick=1),
                                  yaxis = dict(title='# of Collisions', dtick=500, domain = [0,0.5]),
                                  yaxis2 = dict(title='Total Snow (cm)', domain = [0.55,1.0]),
                                  xaxis2 = dict(dtick=1, anchor = 'y2')
                                 )
              )
    iplot(fig)
    
for m in range(2014,2018):
    printSnowInYear(m)

In [18]:
df_avg = wt1417.groupby(["Year","Month"]).mean()
df_avg["Year"] = [x[0] for x in df_avg.index]
df_avg["Month"] = [x[1] for x in df_avg.index]

df_sum = wt1417.groupby(["Year","Month"]).sum()
df_sum["Year"] = [x[0] for x in df_sum.index]
df_sum["Month"] = [x[1] for x in df_sum.index]

def printSnowOnGrndInYear(n):
    monthCase = go.Bar(x = df_sum[df_sum["Year"]==n]["Month"],
                       y = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       text = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       textposition = 'auto',
                       opacity = 0.6,
                       name = 'Number of Collisions')
    total_snowOnGrnd = go.Scatter(x = df_sum[df_sum["Year"]==n]["Month"],
                         y = df_sum[df_sum["Year"]==n]["Snow on Grnd (cm)"],
                         mode = 'lines',
                         name = 'Monthly Total Snow on Ground',
                         xaxis = 'x2', 
                         yaxis = 'y2')
    fig = dict(data = [monthCase, total_snowOnGrnd], 
               layout = go.Layout(title = "Snow on Ground Analysis (" + str(n) + ")",
                                  xaxis = dict(title='Month', dtick=1),
                                  yaxis = dict(title='# of Collisions', dtick=500, domain = [0,0.5]),
                                  yaxis2 = dict(title='Snow on Grnd (cm)', domain = [0.55,1.0]),
                                  xaxis2 = dict(dtick=1, anchor = 'y2')
                                 )
              )
    iplot(fig)
    
for m in range(2014,2018):
    printSnowOnGrndInYear(m)

**Brief Conclusion:**
* The curve of monthly snowfall and snow on ground is similar to the trend of monthly collisions. 
**Hypothesis:**
* Snowy weather causes collisions from the perspective of reducing visibility and grip. 

In [15]:
df_avg = wt1417.groupby(["Year","Month"]).mean()
df_avg["Year"] = [x[0] for x in df_avg.index]
df_avg["Month"] = [x[1] for x in df_avg.index]

df_sum = wt1417.groupby(["Year","Month"]).sum()
df_sum["Year"] = [x[0] for x in df_sum.index]
df_sum["Month"] = [x[1] for x in df_sum.index]

def printPrecipInYear(n):
    monthCase = go.Bar(x = df_sum[df_sum["Year"]==n]["Month"],
                       y = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       text = df_sum[df_sum["Year"]==n]["Number of Collisions"],
                       textposition = 'auto',
                       opacity = 0.6,
                       name = 'Number of Collisions')
    total_precip = go.Scatter(x = df_sum[df_sum["Year"]==n]["Month"],
                         y = df_sum[df_sum["Year"]==n]["Total Precip (mm)"],
                         mode = 'lines',
                         name = 'Monthly Total Precip',
                         xaxis = 'x2', 
                         yaxis = 'y2')
    fig = dict(data = [monthCase, total_precip], 
               layout = go.Layout(title = "Precip Analysis (" + str(n) + ")",
                                  xaxis = dict(title='Month', dtick=1),
                                  yaxis = dict(title='# of Collisions', dtick=500, domain = [0,0.5]),
                                  yaxis2 = dict(title='Total Precip (mm)', domain = [0.55,1.0]),
                                  xaxis2 = dict(dtick=1, anchor = 'y2')
                                 )
              )
    iplot(fig)
    
for m in range(2014,2018):
    printPrecipInYear(m)

**Brief Conclusion:**
* No clear relationship between monthly precipitation and collisions

### 2.4 The day in a year with the largest number of collisions

**In 2014:**

In [43]:
maxColl2014 = wt1417[wt1417["Year"]==2014].sort_values(["Number of Collisions"], ascending=[0])
maxColl2014.iloc[[0]]

,Date,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Number of Collisions,Year,Month,Day in month
2,2014-01-03,-21.0,-28.1,-24.6,0.0,0.3,0.0,29,151,2014,1,3


**In 2015:**

In [44]:
maxColl2015 = wt1417[wt1417["Year"]==2015].sort_values(["Number of Collisions"], ascending=[0])
maxColl2015.iloc[[0]]

,Date,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Number of Collisions,Year,Month,Day in month
397,2015-02-02,-14.3,-20.4,-17.4,0.0,3.0,1.5,14,160,2015,2,2


**In 2016:**

In [45]:
maxColl2016 = wt1417[wt1417["Year"]==2016].sort_values(["Number of Collisions"], ascending=[0])
maxColl2016.iloc[[0]]

,Date,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Number of Collisions,Year,Month,Day in month
777,2016-02-17,0.1,-11.2,-5.6,0.0,0.4,0.4,40,152,2016,2,17


**In 2017:**

In [46]:
maxColl2017 = wt1417[wt1417["Year"]==2017].sort_values(["Number of Collisions"], ascending=[0])
maxColl2017.iloc[[0]]

,Date,Max Temp (°C),Min Temp (°C),Mean Temp (°C),Total Rain (mm),Total Snow (cm),Total Precip (mm),Snow on Grnd (cm),Number of Collisions,Year,Month,Day in month
1178,2017-03-24,1.0,-4.0,-1.5,6.0,7.0,12.6,17,134,2017,3,24


**Brief Conclusion:**
* The day in a year with the largest number of collisions are in winter or close to winter. 
* low temperature, and with snow

**Hypothesis:**
* It still supports the hypothesis about weather: low temperature and snowy weather leads to collisions

### 2.5 The location of collisions in Ottawa (on the date with the largest number of collisions)

In [103]:
dayOf2014 = pd.to_datetime('2014/01/03', format = '%Y/%m/%d')
dayOf2015 = pd.to_datetime('2015/02/02', format = '%Y/%m/%d')
dayOf2016 = pd.to_datetime('2016/02/17', format = '%Y/%m/%d')
dayOf2017 = pd.to_datetime('2015/03/24', format = '%Y/%m/%d')

df_dayOf2014 = df_csv1417[df_csv1417["Date"] == dayOf2014]
lat_lons2014 = [[row['latitude'], row['longitude']] for index, row in df_dayOf2014.iterrows()]
lat2014 = [lat for [lat, lon] in lat_lons]
lon2014 = [lon for [lat, lon] in lat_lons]

center2014 = [((np.min(lat2014) + np.max(lat2014))/2), ((np.min(lon2014) + np.max(lon2014))/2)]

in2014 = folium.Map(location=center2014,
                    zoom_start=10,
                    tiles='OpenStreetMap'
                   )

plugin = folium.plugins.MarkerCluster(locations = lat_lons2014)
plugin.add_to(in2014)

in2014

In [104]:
df_dayOf2015 = df_csv1417[df_csv1417["Date"] == dayOf2015]
lat_lons2015 = [[row['latitude'], row['longitude']] for index, row in df_dayOf2015.iterrows()]
lat2015 = [lat for [lat, lon] in lat_lons]
lon2015 = [lon for [lat, lon] in lat_lons]

center2015 = [((np.min(lat2015) + np.max(lat2015))/2), ((np.min(lon2015) + np.max(lon2015))/2)]

in2015 = folium.Map(location=center2015,
                    zoom_start=10,
                    tiles='OpenStreetMap'
                   )

plugin = folium.plugins.MarkerCluster(locations = lat_lons2015)
plugin.add_to(in2015)

in2015

In [108]:
df_dayOf2016 = df_csv1417[df_csv1417["Date"] == dayOf2016]
lat_lons2016 = [[row['latitude'], row['longitude']] for index, row in df_dayOf2016.iterrows()]
lat2016 = [lat for [lat, lon] in lat_lons]
lon2016 = [lon for [lat, lon] in lat_lons]

center2016 = [((np.min(lat2016) + np.max(lat2016))/2), ((np.min(lon2016) + np.max(lon2016))/2)]

in2016 = folium.Map(location=center2016,
                    zoom_start=10,
                    tiles='OpenStreetMap'
                   )

plugin = folium.plugins.MarkerCluster(locations = lat_lons2016)
plugin.add_to(in2016)

in2016

In [109]:
df_dayOf2017 = df_csv1417[df_csv1417["Date"] == dayOf2016]
lat_lons2017 = [[row['latitude'], row['longitude']] for index, row in df_dayOf2017.iterrows()]
lat2017 = [lat for [lat, lon] in lat_lons]
lon2017 = [lon for [lat, lon] in lat_lons]

center2017 = [((np.min(lat2017) + np.max(lat2017))/2), ((np.min(lon2017) + np.max(lon2017))/2)]

in2017 = folium.Map(location=center2017,
                    zoom_start=10,
                    tiles='OpenStreetMap'
                   )

plugin = folium.plugins.MarkerCluster(locations = lat_lons2017)
plugin.add_to(in2017)

in2017